In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from catboost import CatBoostClassifier
import datetime

In [2]:
lfts_train = pd.read_csv('C:\\AADeloitte\\learning\\Boom\\Python Practise files\\LFTS\\train.csv')
lfts_test = pd.read_csv('C:\\AADeloitte\\learning\\Boom\\Python Practise files\\LFTS\\test.csv')

lfts_train['source'], lfts_test['source'] = 'train', 'test'

data = pd.concat([lfts_train, lfts_test], axis=0, sort=False)

In [3]:
# print(data.loan_default.value_counts(dropna= False).div(len(lfts_train)).mul(100))

# print(data.shape)

# print(data.columns)

In [4]:
data.columns = data.columns.str.replace(".", "_")

In [5]:
data.columns

Index(['UniqueID', 'disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID', 'Date_of_Birth',
       'Employment_Type', 'DisbursalDate', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS_SCORE',
       'PERFORM_CNS_SCORE_DESCRIPTION', 'PRI_NO_OF_ACCTS', 'PRI_ACTIVE_ACCTS',
       'PRI_OVERDUE_ACCTS', 'PRI_CURRENT_BALANCE', 'PRI_SANCTIONED_AMOUNT',
       'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_ACTIVE_ACCTS',
       'SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE', 'SEC_SANCTIONED_AMOUNT',
       'SEC_DISBURSED_AMOUNT', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT',
       'NEW_ACCTS_IN_LAST_SIX_MONTHS', 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS',
       'AVERAGE_ACCT_AGE', 'CREDIT_HISTORY_LENGTH', 'NO_OF_INQUIRIES',
       'loan_default', 'source'],
      dtype='object')

In [6]:
# table = pd.pivot_table(data, index='Employment.Type', values='asset_cost', aggfunc='mean')

# def fill_employment(x):
    
#     if x > table.loc['Salaried'][0]:
#         return 'Salaried'
#     else:
#         return 'Self employed'

# data['Employment.Type'].fillna(data['asset_cost'].apply(lambda x: fill_employment(x)), inplace=True)

In [7]:
print(data.Employment_Type.value_counts(dropna=False))
data.Employment_Type.fillna('Unfilled', inplace=True)
print(data.Employment_Type.value_counts(dropna=False))

Self employed    187429
Salaried         147013
NaN               11104
Name: Employment_Type, dtype: int64
Self employed    187429
Salaried         147013
Unfilled          11104
Name: Employment_Type, dtype: int64


In [8]:
# branch_man_defaults = lfts_train.groupby(['branch_id', 'manufacturer_id'])['loan_default'].sum()

In [9]:
# data.join(branch_man,on=['branch_id', 'manufacturer_id'], rsuffix='brn_man_default')

In [10]:
#using cut to create bin from credit scores

data.PERFORM_CNS_SCORE = pd.cut(data.PERFORM_CNS_SCORE, bins=5, labels=['Slim', 'Below Average', 'Average', 'Above Average', 'Great'])
data.PERFORM_CNS_SCORE.value_counts(dropna=False)

Slim             192802
Great             67263
Above Average     57754
Below Average     14777
Average           12950
Name: PERFORM_CNS_SCORE, dtype: int64

In [11]:
#count of defaults by branch
branch_id_defaut_count = dict(data[data.loan_default == 1.0].groupby('branch_id')['loan_default'].count())
data['branchwise_defaults'] = data.branch_id.map(branch_id_defaut_count)
data.branchwise_defaults.fillna(0, inplace=True)

#count of defaults by manufacture
manufacture_id_count = dict(data[data['loan_default'] == 1.0].groupby('manufacturer_id')['loan_default'].count())
data['manufacturewise_defaults'] = data['manufacturer_id'].map(manufacture_id_count)
data.manufacturewise_defaults.fillna(0, inplace=True)

#count of defaults by pincode
pincode_default_count = dict(data[data['loan_default'] == 1.0].groupby('Current_pincode_ID')['loan_default'].count())
data['pincodewise_defaults'] = data['Current_pincode_ID'].map(pincode_default_count)
data.pincodewise_defaults.fillna(0, inplace=True)

#statewise defaults
state_default_count = dict(data[data['loan_default'] == 1.0].groupby('State_ID')['loan_default'].count())
data['statewise_defaults'] = data['State_ID'].map(state_default_count)
data.statewise_defaults.fillna(0, inplace=True)

#employeewise defaults
employee_default_count = dict(data[data['loan_default'] == 1.0].groupby('Employee_code_ID')['loan_default'].count())
data['employeewise_defaults'] = data['Employee_code_ID'].map(employee_default_count)
data.employeewise_defaults.fillna(0, inplace=True)


#supplier defaults
supplier_default_count = dict(data[data['loan_default'] == 1.0].groupby('supplier_id')['loan_default'].count())
data['supplierwise_defaults'] = data['supplier_id'].map(supplier_default_count)
data.supplierwise_defaults.fillna(0, inplace=True)

In [12]:
#creating average of all ID mean variables

branch_default_mean = data.loc[data.loan_default == 1].groupby('branch_id')['loan_default'].count().mean()
supplier_default_mean = data.loc[data.loan_default == 1].groupby('supplier_id')['loan_default'].count().mean()
manufacturer_default_mean = data.loc[data.loan_default == 1].groupby('manufacturer_id')['loan_default'].count().mean()
pincode_default_mean = data.loc[data.loan_default == 1].groupby('Current_pincode_ID')['loan_default'].count().mean()
State_default_mean = data.loc[data.loan_default == 1].groupby('State_ID')['loan_default'].count().mean()
employee_default_mean = data.loc[data.loan_default == 1].groupby('Employee_code_ID')['loan_default'].count().mean()

In [13]:
# data.columns

In [14]:
#function to return above average 

def create_bool(x, mean):
    if x > mean:
        return 1
    else:
        return 0

In [15]:
data['branch_abv_default'] = data.branchwise_defaults.apply(lambda x: create_bool(x, branch_default_mean))
data['supplier_abv_default'] = data.supplierwise_defaults.apply(lambda x: create_bool(x, supplier_default_mean))
data['manufacture_abv_default'] = data.manufacturewise_defaults.apply(lambda x: create_bool(x, manufacturer_default_mean))
data['pincode_abv_default'] = data.pincodewise_defaults.apply(lambda x: create_bool(x, pincode_default_mean))
data['state_abv_default'] = data.statewise_defaults.apply(lambda x: create_bool(x, State_default_mean))
data['employee_abv_default'] = data.employeewise_defaults.apply(lambda x: create_bool(x, employee_default_mean))

In [16]:
# for i in data.columns[data.columns.str.contains('abv')]:
#     print(i)
#     print(data[i].isna().sum())
#     print(data[i].value_counts())
#     print('~~~~~~~~~~~~~~~~~~~')

In [17]:
# for i in data.columns[data.columns.str.contains('abv_default')]:
#     print(data[i].value_counts(dropna=False))
#     print(data[i].value_counts(dropna=False).sum())
#     print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

In [18]:
#creating the 

data.Date_of_Birth.head()

data['Age'] = 2018 - pd.to_numeric((str(19) + data.Date_of_Birth.str.rpartition('-')[2]))

# data.loc[data.Age == 118, 'Age'] = 18
data.Age.replace(118, 18, inplace=True)

In [19]:
print(data.Age.min(), data.Age.max())

18 69


In [20]:
def get_agegroup(x):
    if x >= 18 and x < 30:
        return '18-29'
    elif x >= 30 and x < 40:
        return '30-39'
    elif x >= 40 and x < 50:
        return '40-49'
    elif x >= 50 and x < 60:
        return '50-59'
    elif x >= 60 and x < 70:
        return '60-69'
    elif x >= 70 and x < 80:
        return '70-79'
    else:
        return '80+'

data['Age_group'] = data.Age.apply(lambda x: get_agegroup(x))

data.Age_group.value_counts()

18-29    135454
30-39    109870
40-49     70517
50-59     27397
60-69      2308
Name: Age_group, dtype: int64

In [21]:
data.DisbursalDate.head(10)

0    03-08-18
1    26-09-18
2    01-08-18
3    26-10-18
4    26-09-18
5    19-09-18
6    23-09-18
7    16-09-18
8    05-09-18
9    16-09-18
Name: DisbursalDate, dtype: object

In [22]:
data['DisbursalDate'] = pd.to_datetime(data.DisbursalDate, dayfirst=True, format='%d-%m-%y')

data['Disbursal_day'] = data.DisbursalDate.dt.day
data['Disbursal_weekday'] = data.DisbursalDate.dt.weekday
data['Disbursal_week'] = data.DisbursalDate.dt.week
data['Disbursal_month'] = data.DisbursalDate.dt.month
data['Disbursal_quarter'] = data.DisbursalDate.dt.quarter

In [24]:
# data[data.loan_default == 0].groupby('Disbursal_day')['loan_default'].count().plot.barh()

In [25]:
data.loc[data['PERFORM_CNS_SCORE_DESCRIPTION'].str.contains(r'No'), 'PERFORM_CNS_SCORE_DESCRIPTION'] = 'No Information'

data.loc[data['PERFORM_CNS_SCORE_DESCRIPTION'].str.contains(r'Very Low'), 'PERFORM_CNS_SCORE_DESCRIPTION'] = 'Lowest Risk'

data.loc[data['PERFORM_CNS_SCORE_DESCRIPTION'].str.contains(r'Low Risk'), 'PERFORM_CNS_SCORE_DESCRIPTION'] = 'Low Risk'

data.loc[data['PERFORM_CNS_SCORE_DESCRIPTION'].str.contains(r'Medium Risk'), 'PERFORM_CNS_SCORE_DESCRIPTION'] = 'Medium Risk'

data.loc[data['PERFORM_CNS_SCORE_DESCRIPTION'].str.contains(r'Very High'), 'PERFORM_CNS_SCORE_DESCRIPTION'] = 'Highest Risk'

data.loc[data['PERFORM_CNS_SCORE_DESCRIPTION'].str.contains(r'High Risk'), 'PERFORM_CNS_SCORE_DESCRIPTION'] = 'High Risk'

In [26]:
data.PERFORM_CNS_SCORE_DESCRIPTION.value_counts(dropna=False)

No Information    192802
Lowest Risk        75721
Low Risk           26364
Medium Risk        18402
High Risk          17667
Highest Risk       14590
Name: PERFORM_CNS_SCORE_DESCRIPTION, dtype: int64

In [27]:
data['AVERAGE_ACCT_AGE'][:5].str.split()

0     [0yrs, 0mon]
1    [1yrs, 11mon]
2     [0yrs, 0mon]
3     [0yrs, 8mon]
4     [0yrs, 0mon]
Name: AVERAGE_ACCT_AGE, dtype: object

In [28]:
data['Acct_age_years'] = [int(re.sub(pattern=r'yrs', repl='', string=x))*12 for x in data.AVERAGE_ACCT_AGE.str.rpartition(' ')[0]]
data['Acct_age_months'] = [int(re.sub(pattern=r'mon', repl='', string= x)) for x in data.AVERAGE_ACCT_AGE.str.rpartition(' ')[2]]

In [29]:
data['Acct_age_in_months'] = data['Acct_age_years'].add(data.Acct_age_months)

In [30]:
def get_year(x):
    
    if x > 0 and x <=12:
        return 'Below 1 year'
    elif x > 12 and x <= 36:
        return 'Between 1 and 3 years'
    elif x > 36 and x <= 60:
        return 'Betweem 3 and 5 years'
    elif x > 60 and x < 120:
        return 'Between 5 to 10 years'
    elif x > 120 and x < 180:
        return 'Between 10 and 15 years'
    elif x > 180:
        return 'Above 15 years'
    else:
        return 'Zero'

In [31]:
data['Acct_year_cat'] = data.Acct_age_in_months.apply(lambda x: get_year(x))
data.Acct_year_cat.value_counts(dropna=False)

Zero                       177502
Below 1 year                79802
Between 1 and 3 years       69935
Betweem 3 and 5 years       13239
Between 5 to 10 years        4583
Between 10 and 15 years       460
Above 15 years                 25
Name: Acct_year_cat, dtype: int64

In [32]:
data['Cred_age_years'] = [int(re.sub(pattern=r'yrs', repl='', string=x))*12 for x in data.CREDIT_HISTORY_LENGTH.str.rpartition(' ')[0]]
data['Cred_age_months'] = [int(re.sub(pattern=r'mon', repl='', string= x)) for x in data.CREDIT_HISTORY_LENGTH.str.rpartition(' ')[2]]

In [33]:
data['Credit_history_tenure'] = data.Cred_age_years + data.Cred_age_months

In [34]:
data['Credit_history_tenure_cat'] = data.Credit_history_tenure.apply(lambda x: get_year(x))

In [35]:
# data[data.loan_default == 1].groupby('Credit_history_tenure_cat')['loan_default'].count().plot.barh()

In [36]:
data['TOT_NO_OF_ACCTS'] = data.PRI_NO_OF_ACCTS + data.SEC_NO_OF_ACCTS
data['TOT_ACTIVE_ACCTS'] = data.PRI_ACTIVE_ACCTS + data.SEC_ACTIVE_ACCTS
data['TOT_OVERDUE_ACCTS'] = data.PRI_OVERDUE_ACCTS + data.SEC_OVERDUE_ACCTS
data['TOT_CURRENT_BALANCE'] = data.PRI_CURRENT_BALANCE + data.SEC_CURRENT_BALANCE
data['TOT_DISBURSED_AMOUNT'] = data.PRI_DISBURSED_AMOUNT + data.SEC_DISBURSED_AMOUNT
data['TOT_SANCTIONED_AMOUNT'] = data.PRI_SANCTIONED_AMOUNT + data.SEC_SANCTIONED_AMOUNT
data['TOT_INSTAL_AMT'] = data.PRIMARY_INSTAL_AMT + data.SEC_INSTAL_AMT

In [37]:
data['PRI_MINUS_SEC_NO_OF_ACCTS'] = data.PRI_NO_OF_ACCTS - data.SEC_NO_OF_ACCTS
data['PRI_MINUS_SEC_ACTIVE_ACCTS'] = data.PRI_ACTIVE_ACCTS - data.SEC_ACTIVE_ACCTS
data['PRI_MINUS_SEC_OVERDUE_ACCTS'] = data.PRI_OVERDUE_ACCTS - data.SEC_OVERDUE_ACCTS
data['PRI_MINUS_SEC_CURRENT_BALANCE'] = data.PRI_CURRENT_BALANCE - data.SEC_CURRENT_BALANCE
data['PRI_MINUS_SEC_DISBURSED_AMOUNT'] = data.PRI_DISBURSED_AMOUNT - data.SEC_DISBURSED_AMOUNT
data['PRI_MINUS_SEC_SANCTIONED_AMOUNT'] = data.PRI_SANCTIONED_AMOUNT - data.SEC_SANCTIONED_AMOUNT
data['PRI_MINUS_SEC_INSTAL_AMT'] = data.PRIMARY_INSTAL_AMT - data.SEC_INSTAL_AMT

In [38]:
dis_age_mean = dict(data.groupby('Age_group')['disbursed_amount'].mean())

In [39]:
data['disbursed_mean_by_age'] = data.Age_group.map(dis_age_mean)

In [40]:
data['ltv_cat'] = pd.cut(data.ltv, bins=3, labels=['Low', 'Medium', 'High'])

In [41]:
data['disbursed_by_asset_cost'] = (data.disbursed_amount.div(data['asset_cost'])*100)

In [42]:
data['disbursed_mean_by_branch'] = data.groupby('branch_id')['disbursed_amount'].mean()

In [43]:
def get_overdue(x):
    if x > 0 and x <= 2:
        return 'Low'
    elif x > 2 and x <= 6:
        return 'Medium'
    elif x > 6 and x <= 11:
        return "High"
    elif x > 11 and x <= 19:
        return "Alarming"
    elif x ==0:
        return 'No defaults'
    else:
        return "Highly Risky/Fraud"

In [44]:
data['TOT_OVERDUE_ACCTS_CAT'] = data.TOT_OVERDUE_ACCTS.apply(get_overdue)

data['PRI_OVERDUE_ACCTS_CAT'] = data.PRI_OVERDUE_ACCTS.apply(get_overdue)

data['SEC_OVERDUE_ACCTS_CAT'] = data.SEC_OVERDUE_ACCTS.apply(get_overdue)

In [45]:
def get_loan_brackets(x):
    if x >= 0 and x <= 7:
        return "Low"
    elif x > 7 and x <= 23:
        return "Medium"
    elif x > 23 and x <= 48:
        return "High"
    elif x > 48 and x <= 90:
        return "Alarming"
    else:
        return "Fraud"

In [46]:
data['TOT_NO_OF_ACCTS_CAT'] = data['TOT_NO_OF_ACCTS'].apply(lambda x: get_loan_brackets(x))
data['TOT_NO_OF_ACCTS_CAT'].value_counts(dropna=False)

Low         313342
Medium       28995
High          2875
Alarming       299
Fraud           35
Name: TOT_NO_OF_ACCTS_CAT, dtype: int64

In [47]:
TOT_Bool = (data['TOT_NO_OF_ACCTS'] != 0) & (data['TOT_ACTIVE_ACCTS'] != 0)

In [48]:
data.loc[TOT_Bool, 'percentage_accts_active'] = data.loc[TOT_Bool, 'TOT_ACTIVE_ACCTS'].div(data.loc[TOT_Bool, 'TOT_NO_OF_ACCTS'])
data['percentage_accts_active'].fillna(0, inplace=True)

In [49]:
def perc_closed(x):
    if x >= 0.0 and x < 0.25:
        return 'Below 25'
    elif x >= 0.25 and x < 0.50:
        return 'Between 25 and 50'
    elif x >= 0.50 and x < 0.75:
        return 'Between 50 and 75'
    elif x >= 0.75 and x <= 1.0:
        return 'Above 75'
    else:
        return 'Above 75'

In [50]:
data['percentage_accts_active'] = data.percentage_accts_active.apply(lambda x: perc_closed(x))

In [51]:
data['tot_accts_closed'] = data.TOT_NO_OF_ACCTS.sub(data.TOT_ACTIVE_ACCTS)

In [52]:
tot_slice = (data.TOT_NO_OF_ACCTS != 0) & (data.tot_accts_closed != 0)

In [53]:
data.loc[tot_slice, 'percentage_accts_closed'] = data.loc[tot_slice, 'tot_accts_closed'].div(data.loc[tot_slice, 'TOT_NO_OF_ACCTS'])
data['percentage_accts_closed'].fillna(0, inplace=True)

In [54]:
data['percentage_accts_closed'] = data.percentage_accts_closed.apply(lambda x: perc_closed(x))

In [55]:
overdue_slice = (data['TOT_NO_OF_ACCTS'] != 0) & (data['TOT_OVERDUE_ACCTS'] != 0)

In [56]:
data.loc[overdue_slice, 'percentage_overdue'] = data.loc[overdue_slice, 'TOT_OVERDUE_ACCTS'].div(data.loc[overdue_slice, 'TOT_NO_OF_ACCTS'])
data.percentage_overdue.fillna(0, inplace=True)

In [57]:
data.percentage_overdue = data.percentage_overdue.apply(lambda x: perc_closed(x))

In [58]:
data.percentage_overdue.value_counts(dropna = False)

Below 25             321473
Between 25 and 50      9546
Above 75               7573
Between 50 and 75      6954
Name: percentage_overdue, dtype: int64

In [59]:
del_slice = (data['NEW_ACCTS_IN_LAST_SIX_MONTHS'] != 0) & (data['DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS'] != 0)

In [60]:
data.loc[del_slice, 'del_new_percentage'] = data.loc[del_slice, 'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS'].div(data.loc[del_slice, 'NEW_ACCTS_IN_LAST_SIX_MONTHS'])
data['del_new_percentage'].fillna(0, inplace=True)

In [61]:
data.del_new_percentage = data.del_new_percentage.apply(lambda x: perc_closed(x))
data.del_new_percentage.value_counts(dropna=False)

Below 25             333568
Above 75               7727
Between 50 and 75      2691
Between 25 and 50      1560
Name: del_new_percentage, dtype: int64

In [62]:
# data.del_new_percentage.value_counts(dropna=False).plot.barh()

In [63]:
def get_enquiries(x):
    if x >= 0 and x < 5:
        return 'Low'
    elif x >= 5 and x < 10:
        return 'Medium'
    elif x >= 10 and x < 15:
        return 'High'
    elif x >= 15 and x < 20:
        return 'Very high'
    else:
        return 'Desperate'

In [64]:
data['NO_OF_INQUIRIES'] = data['NO_OF_INQUIRIES'].apply(lambda x: get_enquiries(x))
data.NO_OF_INQUIRIES.value_counts(dropna=False)

Low          344063
Medium         1311
High            130
Very high        31
Desperate        11
Name: NO_OF_INQUIRIES, dtype: int64

In [65]:
# data.drop(['PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT','PRI.DISBURSED.AMOUNT'
#            , 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS','SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT','SEC.DISBURSED.AMOUNT'
#            , 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT', 'branchwise_defaults','manufacturewise_defaults', 'pincodewise_defaults'
#            , 'statewise_defaults', 'employeewise_defaults', 'supplierwise_defaults', 'Acc_age_yrs', 'Acc_age_mnths', 'Cred_age_yrs', 'Cred_age_mnths'
#            , 'Date.of.Birth', 'DisbursalDate', 'NEW.ACCTS.IN.LAST.SIX.MONTHS','DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AVERAGE.ACCT.AGE'
#            ,'CREDIT.HISTORY.LENGTH', 'TOT.NO.OF.ACCTS', 'TOT.ACTIVE.ACCTS', 'TOT.OVERDUE.ACCTS', 'tot_accts_closed'], axis=1, inplace=True)

#  object_cols = ['Employment.Type', 'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag', 'Driving_flag','Passport_flag'
#                , 'PERFORM_CNS.SCORE.DESCRIPTION', 'NO.OF_INQUIRIES', 'branch_abv_default', 'supplier_abv_default', 'manufacture_abv_default'
#                , 'pincode_abv_default', 'State_abv_default', 'employee_abv_default','Age', 'disbursal_week', 'Loan_tenure_length', 'Credit_history_tenure', 'TOT.NO.OF.ACCTS.CAT'
#                , 'TOT.OVERDUE.ACCTS.CAT', 'percentage_accts_active', 'percentage_accts_closed', 'percentage_overdue'
#                , 'del_new_percentage']

In [66]:
cols_to_drop = ['UniqueID', 'Date_of_Birth', 'DisbursalDate', 'MobileNo_Avl_Flag', 'AVERAGE_ACCT_AGE', 'CREDIT_HISTORY_LENGTH', 'Acct_age_years', 'Acct_age_months'
 , 'Cred_age_years', 'Cred_age_months', 'Age', 'PRI_NO_OF_ACCTS', 'PRI_ACTIVE_ACCTS','PRI_OVERDUE_ACCTS', 'PRI_CURRENT_BALANCE'
                , 'PRI_SANCTIONED_AMOUNT', 'PRI_DISBURSED_AMOUNT', 'SEC_NO_OF_ACCTS', 'SEC_ACTIVE_ACCTS','SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE'
                , 'SEC_SANCTIONED_AMOUNT','SEC_DISBURSED_AMOUNT', 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT']

In [67]:
# data.drop(['PRI_NO_OF_ACCTS', 'PRI_ACTIVE_ACCTS','PRI_OVERDUE_ACCTS', 'PRI_CURRENT_BALANCE', 'PRI_SANCTIONED_AMOUNT', 'PRI_DISBURSED_AMOUNT'
#            , 'SEC_NO_OF_ACCTS', 'SEC_ACTIVE_ACCTS','SEC_OVERDUE_ACCTS', 'SEC_CURRENT_BALANCE', 'SEC_SANCTIONED_AMOUNT','SEC_DISBURSED_AMOUNT'
#            , 'PRIMARY_INSTAL_AMT', 'SEC_INSTAL_AMT'], axis=1, inplace=True)

In [68]:
data.drop(cols_to_drop, axis=1, inplace=True)

In [69]:
print(len(data.columns)-2)
data.columns

69


Index(['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Employment_Type', 'State_ID',
       'Employee_code_ID', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS_SCORE',
       'PERFORM_CNS_SCORE_DESCRIPTION', 'NEW_ACCTS_IN_LAST_SIX_MONTHS',
       'DELINQUENT_ACCTS_IN_LAST_SIX_MONTHS', 'NO_OF_INQUIRIES',
       'loan_default', 'source', 'branchwise_defaults',
       'manufacturewise_defaults', 'pincodewise_defaults',
       'statewise_defaults', 'employeewise_defaults', 'supplierwise_defaults',
       'branch_abv_default', 'supplier_abv_default', 'manufacture_abv_default',
       'pincode_abv_default', 'state_abv_default', 'employee_abv_default',
       'Age_group', 'Disbursal_day', 'Disbursal_weekday', 'Disbursal_week',
       'Disbursal_month', 'Disbursal_quarter', 'Acct_age_in_months',
       'Acct_year_cat', 'Credit_history_tenure', 'Credit_history_tenure_cat',
     

In [70]:
data['TOT_INSTAL_AMT_CAT'] = pd.qcut(data.TOT_INSTAL_AMT, q = [.5, .75, .99, .995, 1.], labels=['Lowest', 'Low', 'Medium', 'High'],duplicates='drop')

In [71]:
categorical_cols = ['branch_id', 'manufacturer_id', 'Employment_Type', 'State_ID','Employee_code_ID', 'Aadhar_flag'
                    , 'PAN_flag', 'VoterID_flag', 'Driving_flag', 'Passport_flag', 'PERFORM_CNS_SCORE_DESCRIPTION','branch_abv_default'
                    , 'supplier_abv_default', 'manufacture_abv_default', 'pincode_abv_default', 'state_abv_default', 'employee_abv_default'
                   , 'Age_group', 'Disbursal_day', 'Disbursal_weekday', 'Disbursal_week', 'Disbursal_month', 'Disbursal_quarter', 'Acct_year_cat'
                    , 'Credit_history_tenure_cat', 'ltv_cat', 'TOT_OVERDUE_ACCTS_CAT','PRI_OVERDUE_ACCTS_CAT', 'SEC_OVERDUE_ACCTS_CAT'
                    , 'TOT_NO_OF_ACCTS_CAT', 'TOT_INSTAL_AMT_CAT']

In [72]:
# scaler = StandardScaler()

# data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

# data[object_cols].head()

# dummies = pd.get_dummies(data[object_cols], drop_first=True)

# df = pd.concat([data, dummies], axis=1)

# df.drop(object_cols, axis=1, inplace=True)

# df.columns

In [74]:
train, test = data[data.source == 'train'], data[data.source == 'test']

In [75]:
train.drop('source', axis=1, inplace=True)

C:\Users\mjazeel\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [76]:
predictors, target = train.drop('loan_default', axis=1), train.loan_default.astype(int)

In [ ]:
# target.astype(int)

In [ ]:
categorical_indices = np.where(predictors.dtypes == 'object')[0]
print(predictors.columns[categorical_indices])
len(categorical_indices)
categorical_indices

In [ ]:
for i in predictors.columns[categorical_indices]:
    print(i)
    print(predictors[i].value_counts(dropna=False))
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

In [77]:
X, y = predictors.values, target.values

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1994, stratify= target)

In [ ]:
# GBC_clf = GradientBoostingClassifier()

# RF_clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

# RF_clf.fit(X_train, y_train)

# RF_pred = RF_clf.predict(X_test)

# accuracy_score(y_test, RF_pred)

# f1_score(y_test, RF_pred)

# confusion_matrix(y_test, RF_pred)

In [80]:
from lightgbm import LGBMClassifier, plot_importance

lgbm_clf = LGBMClassifier()

lgbm_clf.fit(X_train, y_train)

# plt.figure(figsize=(48, 48))
# plot_importance(lgbm_clf)

lgbm_pred = lgbm_clf.predict(X)

accuracy_score(y, lgbm_pred)

f1_score(y, lgbm_pred)

In [ ]:
# from xgboost import XGBClassifier

# xgb_clf = XGBClassifier(n_estimators=500, n_jobs=-1)

# xgb_clf.fit(X_train, y_train)

# xgb_pred = xgb_clf.predict(X_test)

# accuracy_score(y_test, xgb_pred)

# xgb_test = xgb_clf.predict(test)

# sv_clf = SVC()

# sv_clf.fit(X_train, y_train)

# sv_pred = sv_clf.predict(X_test)
# accuracy(y_test, sv_pred)